In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [2]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

kinematic_dict, K, P, L, D, C, g, B, qdd, joint_torque, sys_id_coeff , F_payload = alpha.build_model(root, tip, floating_base=True)
c_parms, m_params, I_params, fv_coeff, fs_coeff, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']
n_joints = kinematic_dict['n_joints']
N = alpha.get_N
D_dot_2C = alpha.get_D_dot_2C
H = alpha.get_total_energy
H_dot = alpha.get_total_power
F_next = alpha.get_forward_simulation
R_base = alpha.get_base_Reaction
Y_id_reg = ca.Function('Y_reg',[q,q_dot,q_dotdot,vec_g,base_pose,world_pose], [alpha.Y])
Y_id_reg.save('arm_id_Y.casadi')

In [3]:
Base_ext_R_vehicle = ca.Function('R_base',[ca.vertcat(*c_parms), ca.vertcat(*m_params),ca.vertcat(*I_params),q,q_dot,q_dotdot,vec_g,world_pose,base_pose,m_p,r_com_body],[R_base])

In [4]:
# # c , cpp or matlab code generation for forward simulation
# Base_ext_R_vehicle.generate("Base_ext_R_vehicle.c")
# os.system("gcc -fPIC -shared Base_ext_R_vehicle.c -o libBase_ext_R_vehicle.so")

In [5]:
# # c , cpp or matlab code generation for forward simulation
# F_next.generate("Mnext.c")
# os.system("gcc -fPIC -shared Mnext.c -o libMnext.so")


# c , cpp or matlab code generation for forward simulation
alpha.F_next_reg.generate("Mnext.c")
os.system("gcc -fPIC -shared Mnext.c -o libMnext.so")



0

In [6]:
L.size(), K.size(), P.size(), D.size(), g.size(), B.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1), (4, 1))

In [7]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               q_dot,
                               base_pose,
                               world_pose ],[K, D , C, g, D_dot_2C, N, P])

In [8]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, np.pi, 0.000, 0.000] #floating base mount
# base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
world_pose0 = [0.0, 0.0, 0, 0, 0, 0]
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])
c_sample = [0, 0, 0,
    0, 0, 0,
    0, 0, 0,
    0, 0, 0]
m_sample = [0.194, 0.429, 0.115, 0.333]
I_sample = [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0]
g_vec_sample = [0, 0, 9.81]
fv_sample = [0.0, 0.0, 0.0, 0.0]
fs_sample = [0.0, 0.0, 0.0, 0.0]
tau_sample = [0.2, 0.1, 0.3, 0.6]
qdot_sample = [0.21, 0.9, 0.13, 0.14]
qdotdot_sample = [0.0, 0.0, 0.0, 0.0]
r_com_payload_sample = [0.0, 0.0, 0.0]
m_p_sample = 0.0
dt_sample =0.4
kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                        m_sample,
                                                                                                        I_sample,
                                                                                                        g_vec_sample,
    q_sample,
    qdot_sample,
    base_T0,
    world_pose0
    )
rigid_p_sample = ca.vertcat(c_sample, m_sample, I_sample, fv_sample,fs_sample,g_vec_sample,r_com_payload_sample,m_p_sample,base_T0, world_pose0)
x_next0 = alpha.F_next(ca.vertcat(q_sample, qdot_sample,), tau_sample, dt_sample, rigid_p_sample)
kinetic_energy, robot_inertia_matrix, coriolis_matrix, potential_energy, gravity_vector, x_next0

(DM(0.0145657),
 DM(
 [[0.0496372, -1.15084e-09, 4.97255e-08, -0.00134691], 
  [-1.15084e-09, 0.0382637, -0.0192923, -2.29548e-09], 
  [4.97255e-08, -0.0192923, 0.0204958, 2.95497e-19], 
  [-0.00134691, -2.29548e-09, 3.41259e-19, 0.01]]),
 DM(
 [[-0.00277364, 1.28464e-05, -0.000461069, 0.00381492], 
  [-1.28488e-05, 0.000197218, 0.00116813, -0.00104043], 
  [0.000461073, -0.00136535, -3.37292e-19, 0.00104044], 
  [0.00381492, 0.00104043, -0.00104044, 5.75087e-19]]),
 DM(-1.91568),
 DM([-4.42016e-17, -0.548326, -0.0980198, 0]),
 DM([6.25486, 4.77137, 3.89829, 9.60221, 5.5899, 9.80327, 8.31661, 27.7031]))

In [9]:
# # *c_parms, *m_params, *I_params, fv_coeff, fs_coeff, vec_g, r_com_body, m_p, base_pose, world_pose
# p_sample = c_sample+ m_sample+ I_sample+fv_sample+ fs_sample+ g_vec_sample + r_com_body_sample + [m_p_sample] + base_T0 + base_origin
# p_sample

In [10]:
mrc_lump, I_lump  = sys_id_coeff['first_moments'],  sys_id_coeff['inertias_vec6']


mrc_I_i_fun = ca.Function('mrc_I_i', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               q,
                               base_pose,
                               world_pose ], [ca.vertcat(*mrc_lump), ca.vertcat(*I_lump)])

            
m_id_vec = sys_id_coeff['masses_id_syms_vertcat']
m_rci_id_vec = sys_id_coeff['first_moments_id_vertcat']
I_id_vec = sys_id_coeff['inertias_id_vertcat']
fv_id_vec = sys_id_coeff['fv_id_vertcat']
fs_id_vec = sys_id_coeff['fs_id_vertcat']
 
reg_fun = ca.Function('reg', [m_id_vec, m_rci_id_vec, I_id_vec, vec_g, q,q_dot,base_pose,world_pose],[alpha.id_K, alpha.id_D, alpha.id_C, alpha.id_P, alpha.id_g])

mrc_lump_sample , I_lump_sample =  mrc_I_i_fun(c_sample, m_sample, I_sample ,q_sample, base_T0, world_pose0)
id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector = reg_fun(m_sample, mrc_lump_sample, I_lump_sample, g_vec_sample, q_sample, qdot_sample, base_T0, world_pose0)
rigid_p_id_sample = ca.vertcat(m_sample, mrc_lump_sample, I_lump_sample,fv_sample,fs_sample,g_vec_sample,r_com_payload_sample,m_p_sample,base_T0, world_pose0)

x_nextid0 = alpha.F_next_reg(ca.vertcat(q_sample, qdot_sample,), tau_sample, dt_sample, rigid_p_id_sample)
id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector , x_nextid0

(DM(0.0145657),
 DM(
 [[0.0496372, -1.15084e-09, 4.97255e-08, -0.00134691], 
  [-1.15084e-09, 0.0382637, -0.0192923, -2.29548e-09], 
  [4.97255e-08, -0.0192923, 0.0204958, 4.80281e-19], 
  [-0.00134691, -2.29548e-09, 3.4126e-19, 0.01]]),
 DM(
 [[-0.00277364, 1.28464e-05, -0.000461069, 0.00381492], 
  [-1.28488e-05, 0.000197218, 0.00116813, -0.00104043], 
  [0.000461073, -0.00136535, 1.05767e-19, 0.00104044], 
  [0.00381492, 0.00104043, -0.00104044, -7.59628e-21]]),
 DM(-1.91568),
 DM([-4.42016e-17, -0.548326, -0.0980198, 0]),
 DM([6.25486, 4.77137, 3.89829, 9.60221, 5.5899, 9.80327, 8.31661, 27.7031]))

In [11]:
tol = 1e-10
errs = {
    "K": float(ca.norm_inf(kinetic_energy - id_kinetic_energy)),
    "D": float(ca.norm_inf(robot_inertia_matrix - id_robot_inertia_matrix)),
    "C": float(ca.norm_inf(coriolis_matrix - id_coriolis_matrix)),
    "P": float(ca.norm_inf(potential_energy - id_potential_energy)),
    "g": float(ca.norm_inf(gravity_vector - id_gravity_vector)),
    "fsim": float(ca.norm_inf(x_next0 - x_nextid0)),
}
print(errs, {k: (v < tol) for k, v in errs.items()})

{'K': 5.204170427930421e-18, 'D': 6.938893903907228e-18, 'C': 3.2526065174565133e-18, 'P': 0.0, 'g': 1.1102230246251565e-16, 'fsim': 2.149391775674303e-13} {'K': True, 'D': True, 'C': True, 'P': True, 'g': True, 'fsim': True}


In [12]:
J_tip = kinematic_dict["geo_J"][-1]   # 6×n geometric Jacobian at the tool
SYS_ID_FUN = ca.Function('SYS_ID_TAU', [m_id_vec, m_rci_id_vec, I_id_vec, fv_id_vec,fs_id_vec, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, base_pose,world_pose],[alpha.Y@alpha.id_theta + J_tip.T@F_payload])
SYS_ID_Y_FUN = ca.Function('SYS_ID_Y', [vec_g, r_com_body, m_p, q,q_dot, q_dotdot, base_pose,world_pose],[alpha.Y])

SYS_ID_FUN(m_sample, mrc_lump_sample, I_lump_sample, fv_sample, fs_sample, g_vec_sample, r_com_payload_sample, m_p_sample, q_sample, qdot_sample, qdotdot_sample, base_T0, world_pose0)

DM([-9.67528e-05, -0.548145, -0.0990061, 0.00160226])

In [13]:
Inverse_ID_FUN =ca.Function('Inverse_dynamics_tau', [ca.vertcat(*c_parms), ca.vertcat(*m_params), ca.vertcat(*I_params), fv_coeff, fs_coeff,
                                                     vec_g, r_com_body, m_p, q, q_dot, q_dotdot, base_pose, world_pose], [alpha.get_inverse_dynamics])
Inverse_ID_FUN(c_sample,m_sample,I_sample, fv_sample, fs_sample, g_vec_sample, r_com_payload_sample, m_p_sample, q_sample, qdot_sample, qdotdot_sample, base_T0, world_pose0)

DM([-9.67528e-05, -0.548145, -0.0990061, 0.00160226])

In [14]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                            m_sample,
                                                                                                            I_sample,
                                                                                                            g_vec_sample,
        q_sample,
        qdot_sample,
        base_T0,
        world_pose0
        )

    checks = part_properties.is_spd_strict(robot_inertia_matrix) and part_properties.is_skew_symmetric(N_matrix) and np.allclose(D_dot_2C_matrix, N_matrix)
    if not checks:
        raise ValueError(f"check failed for q:{q_sample}")